In [1]:
import pandas as pd
import requests as rq
import numpy as np
import pyarrow
from bs4 import BeautifulSoup as bs
import os
from dotenv import load_dotenv
import time
from datetime import datetime
import sys #permite navegar por el sistema
sys.path.append("../") #solo aplica al soporte
from src.etl import extract as ex

In [2]:
load_dotenv()

True

In [3]:
RUTA_SERVICE = os.getenv("RUTA_SERVICE")
URL_ESCRAPEO_INICIAL = os.getenv("URL_ESCRAPEO_INICIAL")

In [4]:
url_destinos= URL_ESCRAPEO_INICIAL #url destinos totales
sopa_destinos = ex.crear_sopa(url_destinos) #creamos la sopa
#Extraemos todos los contenidos de la clase landing-destination-title
clase_landing_destination_title=sopa_destinos.find_all("div", {"class":"landing-destination-title"})
print(f'Existen {len(clase_landing_destination_title)} destinos diferentes')

Existen 119 destinos diferentes


In [5]:
df_destinos_totales = ex.extracción_destinos_con_url(sopa_destinos) #escrapeo los destinos totales
df_destinos_totales.head()

,nombre_pais,num_viajes_ofertados,url_destino,fecha_escrapeo
0,Albania,1,https://es.tui.com/viajes/albania,2025-04-20
1,Alemania,11,https://es.tui.com/viajes/alemania,2025-04-20
2,Antigua,2,https://es.tui.com/viajes/antiguacaribe,2025-04-20
3,Argentina,23,https://es.tui.com/viajes/argentina,2025-04-20
4,Aruba,3,https://es.tui.com/viajes/aruba,2025-04-20


In [6]:
df_destinos_totales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   nombre_pais           119 non-null    object        
 1   num_viajes_ofertados  119 non-null    int64         
 2   url_destino           119 non-null    object        
 3   fecha_escrapeo        119 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 3.8+ KB


In [7]:
#añado los destinos actuales a los destinos anteriores y guardo el DF con los destinos totales acumulados
ARCHIVO_GUARDAR_ESCRAPEO_DESTINOS = os.getenv("ARCHIVO_GUARDAR_ESCRAPEO_DESTINOS")
if os.path.exists(ARCHIVO_GUARDAR_ESCRAPEO_DESTINOS): #comprueba si existe un fichero ya con destinos
    df_existente_destinos = pd.read_pickle(ARCHIVO_GUARDAR_ESCRAPEO_DESTINOS) #en caso de existir lo importa
    df_combinado_destinos = pd.concat([df_existente_destinos,df_destinos_totales], ignore_index=True) #combina el fichero anterior con el nuevo obtenido en el escrapeo
else:
    df_combinado_destinos=df_destinos_totales #si no existe fichero previo, entonces, crea uno con el obtenido en el escrapeo
df_combinado_destinos.drop_duplicates(inplace=True)
df_combinado_destinos.to_pickle(ARCHIVO_GUARDAR_ESCRAPEO_DESTINOS) #guarda el fichero en formato formato pickle

In [8]:
df_combinado_destinos.shape

(1545, 4)

In [9]:
df_combinado_destinos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1545 entries, 0 to 1544
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   nombre_pais           1545 non-null   object        
 1   num_viajes_ofertados  1545 non-null   int64         
 2   url_destino           1545 non-null   object        
 3   fecha_escrapeo        1545 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 60.4+ KB


In [10]:
df_viajes_totales_destinos, df_continentes, df_opciones = ex.continentes_viajes_totales_destinos_url_y_opciones_1(df_destinos_totales)

 Para Albania en Europa se ha encontrado 1 viajes diferentes
 Para Alemania en Europa se ha encontrado 11 viajes diferentes
 Para Antigua en Caribe se ha encontrado 2 viajes diferentes
 Para Argentina en América se ha encontrado 23 viajes diferentes
 Para Aruba en Caribe se ha encontrado 3 viajes diferentes
 Para Australia en Oceanía se ha encontrado 16 viajes diferentes
 Para Austria en Europa se ha encontrado 14 viajes diferentes
 Para Bahamas en Caribe se ha encontrado 5 viajes diferentes
 Para Barbados en Caribe se ha encontrado 2 viajes diferentes
 Para Bélgica en Europa se ha encontrado 5 viajes diferentes
 Para Belice en América se ha encontrado 1 viajes diferentes
 Para Bolivia en América se ha encontrado 2 viajes diferentes
 Para Botsuana en África se ha encontrado 8 viajes diferentes
 Para Brasil en América se ha encontrado 9 viajes diferentes
 Para Bulgaria en Europa se ha encontrado 4 viajes diferentes
 Para Bután en Asia se ha encontrado 6 viajes diferentes
 Para Cabo Verd

In [11]:
df_viajes_totales_destinos.head()

,pais,nombre_viaje,duracion_viaje,itinerario,precio,url,fecha_escrapeo
0,Albania,Albania y Montenegro,8 Días / 7 Noches,"Tirana, Dürres, Berat, Kruja, Shkodër, Petrova...",1755,https://es.tui.com/viaje/albania-y-montenegro,2025-04-20
1,Alemania,Berlín,4 Días / 3 Noches,Berlín,708,https://viajeonline.es.tui.com/es/idea/1463025...,2025-04-20
2,Alemania,"Fly & Drive Austria, Entre Lagos y Montañas",8 Días / 7 Noches,"Salzburgo, Gmunden, Linz, Melk, Viena, Graz, H...",827,https://viajeonline.es.tui.com/es/idea/7897512...,2025-04-20
3,Alemania,Fly & Drive Carlomagno a su aire,10 Días / 9 Noches,"Bruselas, Durbuy, Lieja, Aquisgrán, Colonia, B...",857,https://viajeonline.es.tui.com/es/idea/8404511...,2025-04-20
4,Alemania,Fly & Drive Tirol al Completo,8 Días / 7 Noches,"sbruck, Cataratas Krimml, Vipiteno, Bresanona,...",894,https://viajeonline.es.tui.com/es/idea/7906930...,2025-04-20


In [12]:
df_viajes_totales_destinos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1058 entries, 0 to 1057
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   pais            1058 non-null   object        
 1   nombre_viaje    1058 non-null   object        
 2   duracion_viaje  1058 non-null   object        
 3   itinerario      1058 non-null   object        
 4   precio          1058 non-null   int64         
 5   url             1058 non-null   object        
 6   fecha_escrapeo  1058 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 58.0+ KB


In [13]:
#añado los viajes actuales a los viajes anteriores y guardo el DF con los viajes totales de todos los destinos acumulados
ARCHIVO_GUARDAR_ESCRAPEO_VIAJES = os.getenv("ARCHIVO_GUARDAR_ESCRAPEO_VIAJES")
if os.path.exists(ARCHIVO_GUARDAR_ESCRAPEO_VIAJES): #comprueba si existe un fichero ya con viajes
    df_existente_viajes_totales= pd.read_pickle(ARCHIVO_GUARDAR_ESCRAPEO_VIAJES)#en caso de existir lo importa
    df_combinado_viajes_totales = pd.concat([df_existente_viajes_totales,df_viajes_totales_destinos], ignore_index=True) #combina el fichero anterior con el nuevo obtenido en el escrapeo
else:
    df_combinado_viajes_totales=df_viajes_totales_destinos  #si no existe fichero previo, entonces, crea uno con el obtenido en el escrapeo
df_combinado_viajes_totales.drop_duplicates(inplace=True)
df_combinado_viajes_totales.to_pickle(ARCHIVO_GUARDAR_ESCRAPEO_VIAJES) #guarda el fichero en formato pickle

In [14]:
df_combinado_viajes_totales.shape

(13475, 7)

In [15]:
df_combinado_viajes_totales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13475 entries, 0 to 13474
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   pais            13475 non-null  object        
 1   nombre_viaje    13475 non-null  object        
 2   duracion_viaje  13475 non-null  object        
 3   itinerario      13475 non-null  object        
 4   precio          13475 non-null  int64         
 5   url             13412 non-null  object        
 6   fecha_escrapeo  13475 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 842.2+ KB


In [16]:
df_continentes.head()

,continente,pais,fecha_escrapeo
0,Europa,Albania,2025-04-20
1,Europa,Alemania,2025-04-20
2,Caribe,Antigua,2025-04-20
3,América,Argentina,2025-04-20
4,Caribe,Aruba,2025-04-20


In [17]:
df_continentes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   continente      119 non-null    object        
 1   pais            119 non-null    object        
 2   fecha_escrapeo  119 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 2.9+ KB


In [18]:
#añado los continentes actuales a los continentes anteriores y guardo el DF con los continentes totales de todos los destinos acumulados
ARCHIVO_GUARDAR_CONTINENTES = os.getenv("ARCHIVO_GUARDAR_CONTINENTES")
if os.path.exists(ARCHIVO_GUARDAR_CONTINENTES): #comprueba si existe un fichero ya con continentes
    df_existente_continentes= pd.read_pickle(ARCHIVO_GUARDAR_CONTINENTES)#en caso de existir lo importa
    df_combinado_continentes = pd.concat([df_existente_continentes,df_continentes], ignore_index=True) #combina el fichero anterior con el nuevo obtenido en el escrapeo
else:
    df_combinado_continentes=df_continentes  #si no existe fichero previo, entonces, crea uno con el obtenido en el escrapeo
df_combinado_continentes.drop_duplicates(inplace=True)
df_combinado_continentes.to_pickle(ARCHIVO_GUARDAR_CONTINENTES) #guarda el fichero en formato pickle

In [19]:
df_combinado_continentes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 725 entries, 0 to 724
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   continente      725 non-null    object        
 1   pais            725 non-null    object        
 2   fecha_escrapeo  725 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 22.7+ KB


In [20]:
df_opciones.head()

,pais,nombre_viaje,opcion,precio,url_opcion,fecha_escrapeo
0,Alemania,Encantos del Rin y Valle del Mosela,Salida desde Madrid,1835,https://es.tui.com/viaje/encantos-del-rin-y-va...,2025-04-20
1,Alemania,Encantos del Rin y Valle del Mosela,Salida desde Barcelona,1930,https://es.tui.com/viaje/encantos-del-rin-y-va...,2025-04-20
2,Alemania,Encantos del Rin y Valle del Mosela,Salida desde Bilbao,1952,https://es.tui.com/viaje/encantos-del-rin-y-va...,2025-04-20
3,Alemania,Lo Mejor del Rin y Países Bajos,Salida desde Madrid,1949,https://es.tui.com/viaje/lo-mejor-del-rin-y-pa...,2025-04-20
4,Alemania,Lo Mejor del Rin y Países Bajos,Salida desde Barcelona,2040,https://es.tui.com/viaje/lo-mejor-del-rin-y-pa...,2025-04-20


In [21]:
df_opciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 823 entries, 0 to 822
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   pais            823 non-null    object        
 1   nombre_viaje    823 non-null    object        
 2   opcion          823 non-null    object        
 3   precio          823 non-null    int64         
 4   url_opcion      823 non-null    object        
 5   fecha_escrapeo  823 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 38.7+ KB


In [22]:
#añado las opciones actuales a las opciones anteriores y guardo el DF con las opciones totales de todos los destinos acumulados
ARCHIVO_GUARDAR_OPCIONES_VIAJES = os.getenv("ARCHIVO_GUARDAR_OPCIONES_VIAJES")
if os.path.exists(ARCHIVO_GUARDAR_OPCIONES_VIAJES): #comprueba si existe un fichero ya con continentes
    df_existente_opciones= pd.read_pickle(ARCHIVO_GUARDAR_OPCIONES_VIAJES)#en caso de existir lo importa
    df_combinado_opciones_viajes = pd.concat([df_existente_opciones,df_opciones], ignore_index=True) #combina el fichero anterior con el nuevo obtenido en el escrapeo
else:
    df_combinado_opciones_viajes=df_opciones  #si no existe fichero previo, entonces, crea uno con el obtenido en el escrapeo
df_combinado_opciones_viajes.drop_duplicates(inplace=True)
df_combinado_opciones_viajes.to_pickle(ARCHIVO_GUARDAR_OPCIONES_VIAJES) #guarda el fichero en formato pickle

In [23]:
df_combinado_opciones_viajes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4101 entries, 0 to 4100
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   pais            4101 non-null   object        
 1   nombre_viaje    4101 non-null   object        
 2   opcion          4101 non-null   object        
 3   precio          4101 non-null   int64         
 4   url_opcion      4101 non-null   object        
 5   fecha_escrapeo  4101 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 224.3+ KB
